Оценив комментарии, полученные в результате ревью, я решила написать дополнительный скипт, в котором восполню недостаток, озвученный в пункте 1. А именно: "Работа с индексами выполнена в лоб: нет индекса - создаём, смотрим. А что было бы, если какие-то из предложенных индексов не создавались?".

В данном ноутбуке я планирую оценить работу каждого из предложенных индексов по отдельности.

При выполнении задания на уровень 1 мною были предложены индексы на следующие поля: `PaymentCategory.Name`,  `AccountType.Name`, `Supplier.ProfitByMaterialAsPayer`, `Supplier.ProfitByMaterialAsPayee`, `PaymentCategory.ProfitByMaterial`, `PaymentCategory.CostByMaterial`, `PaymentCategory.NotInPaymentParticipantProfit`.

Предлагаю изначально скопировать основу из скипта `task_level_1.ipynb`, которая содержит импорт необходимых модулей, подключение к БД, необходимые функции для проведения транзакций. Далее провести транзации INSERT и UPDATE без индексов, получить время их выполнения. Затем предлагаю по очереди создавать предложенные индексы и оценивать время выполнения данных транзакций уже с ним.

In [1]:
import datetime
import random
import pyodbc
from faker import Faker

database = 'PaymentData'
user = 'sa'
password = 'reallyStrongPwd123'
cnxn = pyodbc.connect('DSN='+database+';DATABASE='+database+';UID='+user+';PWD='+ password)
cursor = cnxn.cursor()

faker = Faker()

In [2]:
def gener_date(start_time, end_time):
    return datetime.datetime.fromtimestamp(
            random.randint(int(start_time.timestamp()), int(end_time.timestamp()))
        )

In [3]:
#copy of Payment Insertion block from data_generation.ipynb

def payment_insertion(num_of_lines):
    oid_category = []
    query = "SELECT Oid FROM [dbo].[PaymentCategory]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_category.append(participant[0])

    oid_participant = []
    query = "SELECT Oid FROM [dbo].[PaymentParticipant]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_participant.append(participant[0])

    oid_project = []
    query = "SELECT Oid FROM [dbo].[Project]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_project.append(participant[0])

    for _ in range(num_of_lines):
        amount = random.randint(1, 100000)
        category = random.choice(oid_category)
        check_number = random.randint(10000, 99999)
        comment = faker.text(100)
        date = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now()).isoformat()
        created_date = datetime.datetime.fromtimestamp(
            int(datetime.datetime.now().timestamp())
        ).isoformat()
        is_authorized = random.randint(0, 1)
        justificaton = faker.text(30)
        number = random.randint(10000, 99999)
        optimistic_lock_field = random.randint(1, 3)
        payee, payer = random.sample(oid_participant, 2)
        project = random.choice(oid_project)

        query = f"""INSERT INTO [dbo].[Payment] ([Oid], [Amount], [Category], [Project], [Justification], [Comment], [Date], [Payer], [Payee],
                    [OptimisticLockField], [GCRecord], [CreateDate], [CheckNumber], [IsAuthorized], [Number]) 
                    VALUES (NEWID(), {amount}, '{category}', '{project}', '{justificaton}', '{comment}', '{date}', '{payer}', '{payee}', 
                    {optimistic_lock_field}, Null, '{created_date}', '{check_number}', {is_authorized}, '{number}')"""
        cursor.execute(query)
    cursor.commit()

In [4]:
#need a func for update

def payment_update(num_of_lines):
    oid_project = []
    query = f"""SELECT TOP {num_of_lines} Oid FROM [dbo].[Project]"""
    participants = cursor.execute(query)
    for participant in participants:
        oid_project.append(participant[0])
    
    oid_category = []
    query = "SELECT Oid FROM [dbo].[PaymentCategory]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_category.append(participant[0])

    oid_participant = []
    query = "SELECT Oid FROM [dbo].[PaymentParticipant]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_participant.append(participant[0])

    for project in oid_project:
        amount = random.randint(1, 100000)
        category = random.choice(oid_category)
        check_number = random.randint(10000, 99999)
        comment = faker.text(100)
        justificaton = faker.text(30)
        number = random.randint(10000, 99999)
        payee, payer = random.sample(oid_participant, 2)

        query = f"""UPDATE [dbo].[Payment] SET [Amount] = {amount}, [Category] = '{category}', [Justification] = '{justificaton}',
                [Comment] = '{comment}', [Payer] = '{payer}', [Payee] = '{payee}', [CheckNumber] = {check_number}, [Number] = {number} WHERE [Project] = '{project}'"""
        cursor.execute(query)
    cursor.commit()

На данном этапе скопирована основа из скрипта `task_level_1.ipynb`. Давайте посмотрим, сколько по времени выполняются транзакции без использования индексов.

PS: предлагаю проводить несколько экспериментов, в котором будет меняться количество добавляемых/изменяемых строк. Пусть будем использовать значения 1, 10, 100.

In [5]:
# to clean data
def clean_payment_table():
    query = "DELETE FROM [dbo].[Payment]"
    cursor.execute(query)
    cursor.commit()

In [6]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (без добавления индексов):  0.700305
INSERT 10 строк (без добавления индексов):  0.763733
INSERT 100 строк (без добавления индексов):  2.931243


Видно, что добавление 1 строки от добавления 10 несильно отличается, зато при добавлении 100 строк время выполнения возросло и достигло почти 3 секунды. Тем не менее, видно, что скорость выполнения отдельного INSERT гораздо медленнее, чем выполнение отдельно взятого INSERT'a из пачки INSERT'ов.

Теперь выполним операции с UPDATE:

In [7]:
beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

UPDATE 1 строки (без добавления индексов):  0.331093
UPDATE 10 строк (без добавления индексов):  0.167329
UPDATE 100 строк (без добавления индексов):  1.304057


Странно, что UPDATE 10 строк вышел быстрее, чем UPDATE одной строки. Это могу связать лишь с тем, что при втором и последующих выполнениях одного и того же запроса физические чтения могут уменьшаться, а повторная компиляция может не потребоваться. Из-за этого возникает ситуация, что второй и последующие разы запрос выполняется быстрее чем первый. Причина состоит в кэшировании данных и скомпилированных планах запросов.

Теперь будем рассматривать отдельно работу предложенных индексов. Добавим сначала индекс на поле `PaymentCategory.Name`:

In [8]:
cursor.execute("CREATE NONCLUSTERED INDEX iName_PaymentCategory ON [dbo].[PaymentCategory] (Name)")
cursor.commit()

Оценим, как изменится время выполнения запросов:

In [9]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс PaymentCategory.Name): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс PaymentCategory.Name): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс PaymentCategory.Name): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс PaymentCategory.Name):  0.383225
INSERT 10 строк (индекс PaymentCategory.Name):  0.119888
INSERT 100 строк (индекс PaymentCategory.Name):  0.739377


In [10]:
beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс PaymentCategory.Name): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс PaymentCategory.Name): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс PaymentCategory.Name): ', (stop_time - beginner_time).total_seconds())

UPDATE 1 строки (индекс PaymentCategory.Name):  0.033998
UPDATE 10 строк (индекс PaymentCategory.Name):  0.116717
UPDATE 100 строк (индекс PaymentCategory.Name):  1.719621


Время работы запросов с INSERT улучшилось. Снова наблюдается быстрое выполнение INSERT'a на 10 строк в сравнении за запросом на добавление одной строки. Со 100 строками такого не наблюдается, тк значение все таки на порядок выше и такое явление, как "кэширование данных", не в состоянии скорее всего покрыть выросший объем. А в ситуации с 10-ю запросами, получается, в состоянии.

По поводу UPDATE вижу, что при изменении малого кол-ва строк наблюдается улучшение. При увеличении порядка наблюдается ухудшение.

В целом индекс скорее полезный, чем нет. В конечном счете я бы его оставила.

Удалим созданный индекс, добавим второй (`AccountType.Name`) и проведем те же операции:

In [17]:
cursor.execute("DROP INDEX iName_PaymentCategory ON [dbo].[PaymentCategory]")
cursor.commit()

In [18]:
cursor.execute("CREATE NONCLUSTERED INDEX iName_AccountType ON [dbo].[AccountType] (Name)")
cursor.commit()

In [19]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс AccountType.Name): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс AccountType.Name): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс AccountType.Name): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс AccountType.Name): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс AccountType.Name): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс AccountType.Name): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс AccountType.Name):  0.573669
INSERT 10 строк (индекс AccountType.Name):  0.096858
INSERT 100 строк (индекс AccountType.Name):  0.981719
UPDATE 1 строки (индекс AccountType.Name):  0.020084
UPDATE 10 строк (индекс AccountType.Name):  0.105488
UPDATE 100 строк (индекс AccountType.Name):  1.361697


В некотором смысле результаты даже лучше, чем при добавлении предыдущего индекса, за исключением INSERT 100 строк. Поэтому вердикт: оставить.

Удалим данный индекс, добавим следующий (`Supplier.ProfitByMaterialAsPayer`):

In [20]:
cursor.execute("DROP INDEX iName_AccountType ON [dbo].[AccountType]")
cursor.commit()

In [21]:
cursor.execute("CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayer)")
cursor.commit()

In [22]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс Supplier.ProfitByMaterialAsPayer):  0.258058
INSERT 10 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.130145
INSERT 100 строк (индекс Supplier.ProfitByMaterialAsPayer):  1.0135
UPDATE 1 строки (индекс Supplier.ProfitByMaterialAsPayer):  0.014806
UPDATE 10 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.094439
UPDATE 100 строк (индекс Supplier.ProfitByMaterialAsPayer):  1.365086


Данный индекс сокращает время исполнения запросов INSERT/UPDATE и в сравнении как с начальными данными, так и с предыдущими индексами. Оставить.

Удалим данный индекс, проверим следующий (`Supplier.ProfitByMaterialAsPayee`):

In [23]:
cursor.execute("DROP INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier]")
cursor.commit()

In [24]:
cursor.execute("CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayee)")
cursor.commit()

In [25]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс Supplier.ProfitByMaterialAsPayer):  0.06391
INSERT 10 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.116162
INSERT 100 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.771646
UPDATE 1 строки (индекс Supplier.ProfitByMaterialAsPayer):  0.022103
UPDATE 10 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.100325
UPDATE 100 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.971273


Каждый новый индекс показывает результаты лучшие, чем предыдущий. Уверена, что так происходит из-за ситуации, которую я уже описывала. Сложно конечно таким образом объективно оценивать полезность индексов. Я считаю, имеет смысл просто проверить далее все индексы и посмотреть, сохраняется ли тенденция. И если какой-то индекс будет выбиваться из последовательности, принимать решение о его исключении из набора.

In [26]:
cursor.execute("DROP INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier]")
cursor.commit()

In [27]:
cursor.execute("CREATE NONCLUSTERED INDEX iProfitByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (ProfitByMaterial)")
cursor.commit()

In [28]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс Supplier.ProfitByMaterialAsPayer): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс Supplier.ProfitByMaterialAsPayer):  0.194718
INSERT 10 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.085957
INSERT 100 строк (индекс Supplier.ProfitByMaterialAsPayer):  1.313085
UPDATE 1 строки (индекс Supplier.ProfitByMaterialAsPayer):  0.015509
UPDATE 10 строк (индекс Supplier.ProfitByMaterialAsPayer):  0.14899
UPDATE 100 строк (индекс Supplier.ProfitByMaterialAsPayer):  1.155862


Индекс из тенденции не выбивается, но время на добавление/изменение 100 строк увеличилось. Но оно не самое худшее. Оставить.

In [29]:
cursor.execute("DROP INDEX iProfitByMaterial_PaymentCategory ON [dbo].[PaymentCategory]")
cursor.commit()

cursor.execute("CREATE NONCLUSTERED INDEX iCostByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (CostByMaterial)")
cursor.commit()

In [30]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс PaymentCategory.CostByMaterial): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс PaymentCategory.CostByMaterial): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс PaymentCategory.CostByMaterial): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс PaymentCategory.CostByMaterial): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс PaymentCategory.CostByMaterial): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс PaymentCategory.CostByMaterial): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс PaymentCategory.CostByMaterial):  0.197042
INSERT 10 строк (индекс PaymentCategory.CostByMaterial):  0.073267
INSERT 100 строк (индекс PaymentCategory.CostByMaterial):  0.785569
UPDATE 1 строки (индекс PaymentCategory.CostByMaterial):  0.013668
UPDATE 10 строк (индекс PaymentCategory.CostByMaterial):  0.083515
UPDATE 100 строк (индекс PaymentCategory.CostByMaterial):  0.923495


Из тенденции не выбивается, оставить.

In [31]:
cursor.execute("DROP INDEX iCostByMaterial_PaymentCategory ON [dbo].[PaymentCategory]")
cursor.commit()

cursor.execute("CREATE NONCLUSTERED INDEX iNotInPaymentParticipantProfit_PaymentCategory ON [dbo].[PaymentCategory] (NotInPaymentParticipantProfit)")
cursor.commit()

In [32]:
clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(1)
stop_time = datetime.datetime.now()

print('INSERT 1 строки (индекс PaymentCategory.NotInPaymentParticipantProfit): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(10)
stop_time = datetime.datetime.now()
print('INSERT 10 строк (индекс PaymentCategory.NotInPaymentParticipantProfit): ', (stop_time - beginner_time).total_seconds())

clean_payment_table()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()
print('INSERT 100 строк (индекс PaymentCategory.NotInPaymentParticipantProfit): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(1)
stop_time = datetime.datetime.now()

print('UPDATE 1 строки (индекс PaymentCategory.NotInPaymentParticipantProfit): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(10)
stop_time = datetime.datetime.now()

print('UPDATE 10 строк (индекс PaymentCategory.NotInPaymentParticipantProfit): ', (stop_time - beginner_time).total_seconds())

beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (индекс PaymentCategory.NotInPaymentParticipantProfit): ', (stop_time - beginner_time).total_seconds())

INSERT 1 строки (индекс PaymentCategory.NotInPaymentParticipantProfit):  0.181426
INSERT 10 строк (индекс PaymentCategory.NotInPaymentParticipantProfit):  0.072708
INSERT 100 строк (индекс PaymentCategory.NotInPaymentParticipantProfit):  4.745416
UPDATE 1 строки (индекс PaymentCategory.NotInPaymentParticipantProfit):  0.018332
UPDATE 10 строк (индекс PaymentCategory.NotInPaymentParticipantProfit):  0.127468
UPDATE 100 строк (индекс PaymentCategory.NotInPaymentParticipantProfit):  2.517406


Выбивается из тенденции, показывает худшее время, чем даже при начальном прогоне запросов на 100 строках. Я бы не добавляла.